<a href="https://colab.research.google.com/github/Hadiqapieas/Custom-Model-Architectures/blob/ResNet101/ResNet101c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (Conv2D, BatchNormalization, Activation,
                                     AveragePooling2D, Dense, Flatten,
                                     Dropout, Input, Add)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2


In [2]:
def residual_block(X, num_filters: int, stride: int = 1, kernel_size: int = 3,
                   activation: str = 'relu', bn: bool = True, conv_first: bool = True):
    """
    Residual block with two convolutional layers.

    Parameters
    ----------
    X : Tensor layer
        Input tensor from previous layer
    num_filters : int
        Number of filters for Conv2D layers
    stride : int, default 1
        Stride dimension for Conv2D layers
    kernel_size : int, default 3
        Kernel size for Conv2D layers
    activation: str, default 'relu'
        Activation function to use
    bn: bool, default True
        Use BatchNormalization if True
    conv_first : bool, default True
        Conv-BN-Activation order if True, otherwise BN-Activation-Conv
    """
    # First convolutional layer
    if conv_first:
        X = Conv2D(num_filters, kernel_size=kernel_size, strides=stride,
                   padding='same', kernel_regularizer=l2(1e-4))(X)
        if bn:
            X = BatchNormalization()(X)
        if activation:
            X = Activation(activation)(X)
    else:
        if bn:
            X = BatchNormalization()(X)
        if activation:
            X = Activation(activation)(X)
        X = Conv2D(num_filters, kernel_size=kernel_size, strides=stride,
                   padding='same', kernel_regularizer=l2(1e-4))(X)

    # Second convolutional layer
    X = Conv2D(num_filters, kernel_size=kernel_size, strides=1,
               padding='same', kernel_regularizer=l2(1e-4))(X)
    if bn:
        X = BatchNormalization()(X)
    if activation:
        X = Activation(activation)(X)

    return X


In [3]:
def build_resnet_model(input_shape, num_classes, depth=101):
    """
    Builds a custom ResNet model.

    Parameters
    ----------
    input_shape : tuple
        Shape of the input tensor
    num_classes : int
        Number of output classes
    depth : int, default 101
        Number of layers for ResNet
    """
    num_filters_in = 64
    num_res_block = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    X = residual_block(inputs, num_filters=num_filters_in, conv_first=True)

    for stage in range(3):
        for unit_res_block in range(num_res_block):
            activation = 'relu'
            bn = True
            stride = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if unit_res_block == 0:
                    activation = None
                    bn = False
            else:
                num_filters_out = num_filters_in * 2
                if unit_res_block == 0:
                    stride = 2

            y = residual_block(X, num_filters=num_filters_in, kernel_size=1,
                               stride=stride, activation=activation, bn=bn,
                               conv_first=False)
            y = residual_block(y, num_filters=num_filters_in, conv_first=False)
            y = residual_block(y, num_filters=num_filters_out, kernel_size=1,
                               conv_first=False)
            if unit_res_block == 0:
                X = Conv2D(num_filters_out, kernel_size=1, strides=stride,
                           padding='same', kernel_regularizer=l2(1e-4))(X)
                if bn:
                    X = BatchNormalization()(X)
            X = Add()([X, y])
        num_filters_in = num_filters_out

    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = AveragePooling2D(pool_size=8)(X)
    X = Flatten()(X)
    X = Dense(512, activation='relu')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    outputs = Dense(num_classes, activation='softmax')(X)

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [4]:
!unzip train_dataset.zip

Archive:  train_dataset.zip
   creating: test_dataset/Benign/
  inflating: test_dataset/Benign/WBC-Benign-003.jpg  
  inflating: test_dataset/Benign/WBC-Benign-010.jpg  
  inflating: test_dataset/Benign/WBC-Benign-018.jpg  
  inflating: test_dataset/Benign/WBC-Benign-024.jpg  
  inflating: test_dataset/Benign/WBC-Benign-025.jpg  
  inflating: test_dataset/Benign/WBC-Benign-026.jpg  
  inflating: test_dataset/Benign/WBC-Benign-031.jpg  
  inflating: test_dataset/Benign/WBC-Benign-040.jpg  
  inflating: test_dataset/Benign/WBC-Benign-057.jpg  
  inflating: test_dataset/Benign/WBC-Benign-064.jpg  
  inflating: test_dataset/Benign/WBC-Benign-071.jpg  
  inflating: test_dataset/Benign/WBC-Benign-074.jpg  
  inflating: test_dataset/Benign/WBC-Benign-076.jpg  
  inflating: test_dataset/Benign/WBC-Benign-080.jpg  
  inflating: test_dataset/Benign/WBC-Benign-083.jpg  
  inflating: test_dataset/Benign/WBC-Benign-091.jpg  
  inflating: test_dataset/Benign/WBC-Benign-105.jpg  
  inflating: test_da

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = '/content/train_dataset'
val_dir = '/content/validation_dataset'
test_dir = '/content/test_dataset'

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Reduced from 20
    width_shift_range=0.1,  # Reduced from 0.2
    height_shift_range=0.1,  # Reduced from 0.2
    zoom_range=0.1,  # Reduced from 0.2
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = '/content/train_dataset'
val_dir = '/content/validation_dataset'
test_dir = '/content/test_dataset'

# Create ImageDataGenerators with the target size matching the model's input
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Ensure this matches input shape
    batch_size=16,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)


Found 2278 images belonging to 2 classes.
Found 489 images belonging to 2 classes.
Found 489 images belonging to 2 classes.


In [9]:
# Check GPU memory usage
!nvidia-smi

Mon Jul  8 05:32:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0              27W /  70W |  14075MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
#  enable mixed precision training
from tensorflow.keras import mixed_precision

# Set the global policy to mixed_float16
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


In [14]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Define input shape and number of classes
input_shape = (128, 128, 3)
num_classes = 2  # Assuming binary classification for 'Benign' and 'Malignant'

def build_resnet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    X = Conv2D(32, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Conv2D(32, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = AveragePooling2D(pool_size=2)(X)
    X = Flatten()(X)
    X = Dense(128, activation='relu', dtype='float32')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(X)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Build and compile the model
model = build_resnet_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [15]:
input_shape = (128, 128, 3)
num_classes = len(train_generator.class_indices)  # Get the number of classes from the generator

# Define and build the model
model = build_resnet_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10  # You can increase this based on your requirements
)


# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


142/142 ━━━━━━━━━━━━━━━━━━━━ 47s 188ms/step - accuracy: 0.6741 - loss: 0.7344 - val_accuracy: 0.8479 - val_loss: 0.7192
Epoch 2/10
  1/142 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8750 - loss: 0.4721

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


142/142 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.8750 - loss: 0.4721 - val_accuracy: 0.6667 - val_loss: 1.5674
Epoch 3/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 13s 86ms/step - accuracy: 0.9052 - loss: 0.2795 - val_accuracy: 0.8583 - val_loss: 1.2634
Epoch 4/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.8750 - loss: 0.2331 - val_accuracy: 0.7778 - val_loss: 3.3658
Epoch 5/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 13s 86ms/step - accuracy: 0.9367 - loss: 0.2024 - val_accuracy: 0.1562 - val_loss: 6.4807
Epoch 6/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 116us/step - accuracy: 1.0000 - loss: 0.0719 - val_accuracy: 0.1111 - val_loss: 5.6666
Epoch 7/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 13s 85ms/step - accuracy: 0.9367 - loss: 0.1723 - val_accuracy: 0.8521 - val_loss: 0.8748
Epoch 8/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.8750 - loss: 0.1940 - val_accuracy: 0.7778 - val_loss: 0.9007
Epoch 9/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 20s 84ms/step - accuracy: 0.9327 - loss: 0.1960 - val_accuracy

In [16]:
# Save the model
model.save('custom_resnet_model.h5')
# Load and evaluate
model = tf.keras.models.load_model('custom_resnet_model.h5')


In [17]:
input_shape = (32, 32, 3)  # Example input shape, modify as per your data
num_classes = 10  # Example number of classes, modify as per your dataset

model = build_resnet_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 32, 32, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_14 (Activation)           │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 32, 32, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_15 (Activation)           │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_7                  │ (None, 16, 16, 32)          │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │       1,048,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,060,906 (4.05 MB)

 Trainable params: 1,060,522 (4.05 MB)

 Non-trainable params: 384 (1.50 KB)

In [19]:
import torch
torch.save(model, 'model.pth')